In [23]:
import urllib
import os.path
from textgenrnn import textgenrnn
from bs4 import BeautifulSoup
import pickle
import sys

sys.setrecursionlimit(20000)

In [33]:
pages_file = 'pages.pickle'

def get_poem_pages():
    if os.path.isfile(pages_file):
        with open(pages_file, 'rb') as f: 
            return pickle.load(f)
    pages = []
    for i in range(1, 7):
        index = BeautifulSoup(urllib.request.urlopen('https://www.poemhunter.com/shel-silverstein-2/poems/page-' + str(i)), 'html.parser')
        urls = ['https://www.poemhunter.com' + td.contents[1].get('href') for td in index.find_all('td', attrs={'class': 'title'})]
        for url in urls:
            poem_soup = BeautifulSoup(urllib.request.urlopen(url), 'html.parser')
            pages.append(poem_soup)
    return pages

In [34]:
pages = get_poem_pages()

In [32]:
with open(pages_file, 'wb') as f:  
    pickle.dump(pages, f)

In [35]:
def parse_poems(pages):
    poems = []
    for page in pages:
        poem_lines = page.find('div', attrs={'class': 'KonaBody'}).findChildren('p')[0].contents
        poem_dirty = u'\n'.join([str(line).strip() for line in poem_lines]).strip()
        poem_clean = poem_dirty.replace('\n<br/>\n<br/>\n', '\n\n').replace('\n<br/>', '')
        poems.append(poem_clean)
    return poems

In [36]:
poems = parse_poems(pages)

In [37]:
print(len('\n'.join(poems).split()))

38638


In [ ]:
textgen = textgenrnn()
textgen.train_on_texts(poems, num_epochs=2,  gen_epochs=2)

Training on 196,170 character sequences.
Epoch 1/2
